In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib
import os
from tensorflow.contrib import rnn

tf.reset_default_graph()
tf.set_random_seed(777)

if "DISPLAY" not in os.environ:
    matplotlib.use('TkAgg')

import matplotlib.pyplot as plt

def MinMaxScaler(data):
    numerator = data - np.min(data,0)
    denominator = np.max(data,0)-np.min(data,0)
    return numerator / (denominator+1e-7)

xy = np.genfromtxt('/Users/yeseo/desktop/City_Counted_TaxiMach_Link_Dataset_Full_201501_201611.txt',delimiter = ',',dtype = None)
xy2 = np.genfromtxt('/Users/yeseo/desktop/City_Counted_TaxiMach_Link_Dataset_Full_201704 -201708.txt',delimiter = ',',dtype = None)
test_xy = np.genfromtxt('/Users/yeseo/desktop/City_Counted_TaxiMach_Link_Dataset_Full_201709.txt',delimiter = ',',dtype = None)

xy=np.vstack((xy,xy2))

xy= xy[:,:27]
a=xy[:,:2]
b=xy[:,2:]
b = MinMaxScaler(b)
xy=np.hstack((a,b))

test_xy= test_xy[:,:27]
test_a=test_xy[:,:2]
test_b=test_xy[:,2:]
test_b = MinMaxScaler(test_b)
test_xy=np.hstack((test_a,test_b))




seq_length =6
data_dim =27
hidden_dim = 25
output_dim = 25
learning_rate = 0.01
iterations = 1500

train_size = int(len(xy)*0.7)
train_set = xy
test_set = test_xy


def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range (0,len(time_series)-seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i+seq_length,2:]
     
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)
trainX, trainY = build_dataset(train_set,seq_length)
testX,testY = build_dataset(test_set, seq_length)

X = tf.placeholder(tf.float32,[None, seq_length,data_dim])
Y = tf.placeholder(tf.float32,[None,25])

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
outputs,_states = tf.nn.dynamic_rnn(cell,X,dtype = tf.float32)
Y_pred = tf.contrib.layers.fully_connected(outputs[:,-1], output_dim,activation_fn = None)

loss =tf.reduce_mean(tf.square(Y_pred-Y))
train = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

targets = tf.placeholder(tf.float32,[None,25])
predictions = tf.placeholder(tf.float32,[None,25])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions)))

x1 = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25])
x2 = x1+0.5
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(iterations):
        _, step_loss = sess.run([train,loss],feed_dict={X:trainX, Y:trainY})
        print("[step: {}] loss: {}".format(i,step_loss))
        
    test_predict = sess.run(Y_pred, feed_dict = {X:testX})
    rmse_val = sess.run(rmse, feed_dict={targets: testY,predictions: test_predict})
    print("RMSE: {}".format(rmse_val))
    print("pred: {}".format(test_predict[-1,:]))
    print("real: {}".format(testY[-1,:]))
    
    plt.bar(x1,test_predict[-1,:],label = 'predict',color ='b',width = 0.3)
    plt.bar(x2,testY[-1,:],label = 'real',color ='g',width = 0.3)
    plt.legend()
    plt.show()


[step: 0] loss: 0.24513110518455505
[step: 1] loss: 0.17276914417743683
[step: 2] loss: 0.12625710666179657
[step: 3] loss: 0.09312521666288376
[step: 4] loss: 0.06968091428279877
[step: 5] loss: 0.053251445293426514
[step: 6] loss: 0.04264592379331589
[step: 7] loss: 0.03496498242020607
[step: 8] loss: 0.030584165826439857
[step: 9] loss: 0.02799321338534355
[step: 10] loss: 0.024628156796097755
[step: 11] loss: 0.022028572857379913
[step: 12] loss: 0.0208213422447443
[step: 13] loss: 0.019313616678118706
[step: 14] loss: 0.019049759954214096
[step: 15] loss: 0.018254227936267853
[step: 16] loss: 0.017184969037771225
[step: 17] loss: 0.01622062362730503
[step: 18] loss: 0.015558469109237194
[step: 19] loss: 0.014935644343495369
[step: 20] loss: 0.01468423567712307
[step: 21] loss: 0.013777244836091995
[step: 22] loss: 0.013815595768392086
[step: 23] loss: 0.012686308473348618
[step: 24] loss: 0.012535630725324154
[step: 25] loss: 0.011629439890384674
[step: 26] loss: 0.011381182819604

[step: 212] loss: 0.002201726194471121
[step: 213] loss: 0.002192618791013956
[step: 214] loss: 0.002204346703365445
[step: 215] loss: 0.002189913298934698
[step: 216] loss: 0.002174450783059001
[step: 217] loss: 0.002181145129725337
[step: 218] loss: 0.0021677811164408922
[step: 219] loss: 0.0021801076363772154
[step: 220] loss: 0.0021812915802001953
[step: 221] loss: 0.0021609929390251637
[step: 222] loss: 0.002158929593861103
[step: 223] loss: 0.0021498887799680233
[step: 224] loss: 0.002137118950486183
[step: 225] loss: 0.0021442922297865152
[step: 226] loss: 0.0021445145830512047
[step: 227] loss: 0.002149427542462945
[step: 228] loss: 0.0021615675650537014
[step: 229] loss: 0.0021782589610666037
[step: 230] loss: 0.002191550098359585
[step: 231] loss: 0.0022321788128465414
[step: 232] loss: 0.0022186958231031895
[step: 233] loss: 0.00218926090747118
[step: 234] loss: 0.002158903982490301
[step: 235] loss: 0.0021198547910898924
[step: 236] loss: 0.0021009808406233788
[step: 237] l

[step: 420] loss: 0.00239758612588048
[step: 421] loss: 0.001728215953335166
[step: 422] loss: 0.0015412670327350497
[step: 423] loss: 0.0018960883608087897
[step: 424] loss: 0.0018089775694534183
[step: 425] loss: 0.0016432056436315179
[step: 426] loss: 0.0015898306155577302
[step: 427] loss: 0.0017219826113432646
[step: 428] loss: 0.0017342704813927412
[step: 429] loss: 0.0014805028913542628
[step: 430] loss: 0.001697845058515668
[step: 431] loss: 0.0016107000410556793
[step: 432] loss: 0.0015905429609119892
[step: 433] loss: 0.0014897078508511186
[step: 434] loss: 0.0016139349900186062
[step: 435] loss: 0.0015017091063782573
[step: 436] loss: 0.0015035521937534213
[step: 437] loss: 0.0015191204147413373
[step: 438] loss: 0.0015270574949681759
[step: 439] loss: 0.0014648025389760733
[step: 440] loss: 0.001484710373915732
[step: 441] loss: 0.0014861234230920672
[step: 442] loss: 0.0014567140024155378
[step: 443] loss: 0.0014474596828222275
[step: 444] loss: 0.0014702308690175414
[step

[step: 628] loss: 0.0011163519229739904
[step: 629] loss: 0.0011158664710819721
[step: 630] loss: 0.0011193695245310664
[step: 631] loss: 0.0011196290142834187
[step: 632] loss: 0.0011220424203202128
[step: 633] loss: 0.0011345150414854288
[step: 634] loss: 0.001147749018855393
[step: 635] loss: 0.0011800656793639064
[step: 636] loss: 0.0012180296471342444
[step: 637] loss: 0.0012895247200503945
[step: 638] loss: 0.001318042748607695
[step: 639] loss: 0.0013122381642460823
[step: 640] loss: 0.001217440003529191
[step: 641] loss: 0.0011282660998404026
[step: 642] loss: 0.001106834039092064
[step: 643] loss: 0.0011547461617738008
[step: 644] loss: 0.0012077923165634274
[step: 645] loss: 0.0011985027231276035
[step: 646] loss: 0.0011500578839331865
[step: 647] loss: 0.001108297030441463
[step: 648] loss: 0.001105881412513554
[step: 649] loss: 0.0011329303961247206
[step: 650] loss: 0.0011560850543901324
[step: 651] loss: 0.001146774971857667
[step: 652] loss: 0.0011128948535770178
[step: 

[step: 836] loss: 0.0009719937806949019
[step: 837] loss: 0.0009729458251968026
[step: 838] loss: 0.0009588616085238755
[step: 839] loss: 0.0009473114623688161
[step: 840] loss: 0.0009449687204323709
[step: 841] loss: 0.0009513254626654088
[step: 842] loss: 0.0009592935093678534
[step: 843] loss: 0.0009588542161509395
[step: 844] loss: 0.0009540501050651073
[step: 845] loss: 0.0009487351635470986
[step: 846] loss: 0.0009509147494100034
[step: 847] loss: 0.0009624296799302101
[step: 848] loss: 0.000982364872470498
[step: 849] loss: 0.001013708533719182
[step: 850] loss: 0.0010614466154947877
[step: 851] loss: 0.0011351630091667175
[step: 852] loss: 0.0012285261182114482
[step: 853] loss: 0.0012865023454651237
[step: 854] loss: 0.0012458261335268617
[step: 855] loss: 0.0010767366038635373
[step: 856] loss: 0.0009533072006888688
[step: 857] loss: 0.0009971346007660031
[step: 858] loss: 0.0011093082139268517
[step: 859] loss: 0.0011372251901775599
[step: 860] loss: 0.0011418091598898172
[s

[step: 1042] loss: 0.0008734650327824056
[step: 1043] loss: 0.000909249996766448
[step: 1044] loss: 0.0009301027748733759
[step: 1045] loss: 0.0008669340168125927
[step: 1046] loss: 0.0008586195763200521
[step: 1047] loss: 0.0008893857593648136
[step: 1048] loss: 0.0008591096848249435
[step: 1049] loss: 0.0008470127359032631
[step: 1050] loss: 0.0008739376207813621
[step: 1051] loss: 0.0008648440125398338
[step: 1052] loss: 0.0008496735244989395
[step: 1053] loss: 0.0008604557951912284
[step: 1054] loss: 0.0008561401045881212
[step: 1055] loss: 0.0008393952739425004
[step: 1056] loss: 0.000842866487801075
[step: 1057] loss: 0.0008483512792736292
[step: 1058] loss: 0.000840095744933933
[step: 1059] loss: 0.00083882175385952
[step: 1060] loss: 0.0008463471313007176
[step: 1061] loss: 0.0008444191189482808
[step: 1062] loss: 0.0008378605707548559
[step: 1063] loss: 0.0008404036052525043
[step: 1064] loss: 0.0008421578677371144
[step: 1065] loss: 0.000836246064864099
[step: 1066] loss: 0.0

[step: 1244] loss: 0.0008274300489574671
[step: 1245] loss: 0.0007962771924212575
[step: 1246] loss: 0.0007728859782218933
[step: 1247] loss: 0.0007645292789675295
[step: 1248] loss: 0.0007704406161792576
[step: 1249] loss: 0.0007840858306735754
[step: 1250] loss: 0.0007975559565238655
[step: 1251] loss: 0.0008061972912400961
[step: 1252] loss: 0.0008081580745056272
[step: 1253] loss: 0.000803562521468848
[step: 1254] loss: 0.0007951305597089231
[step: 1255] loss: 0.0007841161568649113
[step: 1256] loss: 0.0007736390689387918
[step: 1257] loss: 0.0007653656648471951
[step: 1258] loss: 0.0007607928127981722
[step: 1259] loss: 0.0007601469988003373
[step: 1260] loss: 0.0007626343867741525
[step: 1261] loss: 0.0007669601473025978
[step: 1262] loss: 0.0007718801498413086
[step: 1263] loss: 0.0007769966614432633
[step: 1264] loss: 0.0007825560169294477
[step: 1265] loss: 0.0007899673655629158
[step: 1266] loss: 0.0008021536632440984
[step: 1267] loss: 0.0008191622328013182
[step: 1268] loss

[step: 1446] loss: 0.0007195600774139166
[step: 1447] loss: 0.0007199971005320549
[step: 1448] loss: 0.0007189259631559253
[step: 1449] loss: 0.0007204316207207739
[step: 1450] loss: 0.0007258732803165913
[step: 1451] loss: 0.0007346459315158427
[step: 1452] loss: 0.0007444541552104056
[step: 1453] loss: 0.000759837799705565
[step: 1454] loss: 0.000781150592956692
[step: 1455] loss: 0.0008100224658846855
[step: 1456] loss: 0.0008414068724960089
[step: 1457] loss: 0.0008669389644637704
[step: 1458] loss: 0.0008787040133029222
[step: 1459] loss: 0.0008606166811659932
[step: 1460] loss: 0.0008169642533175647
[step: 1461] loss: 0.0007602974073961377
[step: 1462] loss: 0.0007215323275886476
[step: 1463] loss: 0.0007170190801844001
[step: 1464] loss: 0.0007415549480356276
[step: 1465] loss: 0.0007687901379540563
[step: 1466] loss: 0.0007821407052688301
[step: 1467] loss: 0.0007702505681663752
[step: 1468] loss: 0.0007435738807544112
[step: 1469] loss: 0.0007166758878156543
[step: 1470] loss: